In [2]:
import numpy as np
import open3d as o3d
# import pptk # works with Python 3.6

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# Load ply file
pc_ply = o3d.io.read_point_cloud("./data/P001 2022-01-25 01_39_54.ply")
print('PLY file loaded')

PLY file loaded


In [4]:
# PLY file info
print(dir(pc_ply))

['HalfEdgeTriangleMesh', 'Image', 'LineSet', 'PointCloud', 'RGBDImage', 'TetraMesh', 'TriangleMesh', 'Type', 'Unspecified', 'VoxelGrid', '__add__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__iadd__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'cluster_dbscan', 'colors', 'compute_convex_hull', 'compute_mahalanobis_distance', 'compute_mean_and_covariance', 'compute_nearest_neighbor_distance', 'compute_point_cloud_distance', 'covariances', 'create_from_depth_image', 'create_from_rgbd_image', 'crop', 'dimension', 'estimate_covariances', 'estimate_normals', 'estimate_point_covariances', 'farthest_point_down_sample', 'get_axis_aligned_bounding_box', 'get_center', 'get_geometry_type', 'get_max_bound', 'get_min_bound', 'get_ori

In [5]:
print('Shape of points', np.asarray(pc_ply.points).shape)
print('Shape of colors', np.asarray(pc_ply.colors).shape)

Shape of points (247847, 3)
Shape of colors (247847, 3)


In [6]:
# visualise
o3d.visualization.draw_geometries([pc_ply], window_name="Original Point Cloud")

In [7]:
# downsampling
voxelSize = 0.01
downsampled = pc_ply.voxel_down_sample(voxelSize)

In [8]:
# outliers
cl, ind = downsampled.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
cleaned_pcd = downsampled.select_by_index(ind)

In [9]:
plane_model, inliers = cleaned_pcd.segment_plane(distance_threshold=0.009, ransac_n=3, num_iterations=10000)
inlier_cloud = cleaned_pcd.select_by_index(inliers)
outlier_cloud = cleaned_pcd.select_by_index(inliers, invert=True)

# Visualize the cleaned point cloud
o3d.visualization.draw_geometries([outlier_cloud], window_name="Cleaned Point Cloud")


In [10]:
# Define your bounding box to clean far away points

points = np.asarray(outlier_cloud.points)

min_x, min_y, min_z = points.min(axis=0)[:3]
max_x, max_y, max_z = points.max(axis=0)[:3]



padding = 0.53
bbox = o3d.geometry.AxisAlignedBoundingBox(min_bound=(padding*min_x, padding*min_y ,padding*min_z ), max_bound=(padding*max_x, padding*max_y, max_z))

# Select points within the bounding box
pcd_in_bbox = outlier_cloud.crop(bbox)

# Visualize
o3d.visualization.draw_geometries([pcd_in_bbox], window_name="Point Cloud within BoundingBox")



In [11]:
# Convert the point cloud to a numpy array
points = np.asarray(pcd_in_bbox.points)

# Define a height threshold
min_x, min_y, min_z = points.min(axis=0)[:3]

height_threshold = min_z  + 0.03  # Adjust this value based on your needs

# Create a mask for points below the height threshold
mask = points[:, 2] > height_threshold

# Select points below the height threshold
subset_pcd = pcd_in_bbox.select_by_index(np.where(mask)[0])

# Visualize
o3d.visualization.draw_geometries([subset_pcd], window_name="Cleaned Point Cloud after DBSCAN Clustering on Subset")




In [54]:
from sklearn.cluster import KMeans

# Convert the point cloud to a numpy array
points = np.asarray(pcd_in_bbox.points)

# Calculate the centroid of the entire point cloud
centroid = np.mean(points, axis=0)

# Define the approximate z-coordinate of the feet
feet_z = min_z  # Adjust this value based on your data

# Shift the centroid to the level of the feet
centroid[2] = feet_z

# Define a tolerance
tolerance = 0.05  # Adjust this value based on your needs

# Create a mask for points that are within the tolerance of the new centroid
mask = np.logical_and(points[:, 2] > (centroid[2] - tolerance), points[:, 2] < (centroid[2] + tolerance))

# Extract the segment of the point cloud
segment_points = points[mask]

# Perform k-means clustering on the segment with 2 clusters (the feet and the floor)
kmeans = KMeans(n_clusters=2, random_state=0).fit(segment_points)

# Find the index of the cluster whose centroid is closer to the new centroid
feet_cluster_index = np.argmin(np.linalg.norm(kmeans.cluster_centers_ - centroid, axis=1))

# Create a mask for points in the segment that belong to the feet cluster
segment_mask = kmeans.labels_ == feet_cluster_index

# Combine the segment mask with the original mask to get a mask for the feet in the original point cloud
mask[mask] = segment_mask

# Select points using the mask
final_pcd = pcd_in_bbox.select_by_index(np.where(~mask)[0])

# Visualize
o3d.visualization.draw_geometries([final_pcd], window_name="Cleaned Point Cloud after Removing Floor Points")


C:\Users\Anthony\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [56]:
# Define the distance thresholds
x_threshold = 0.01  # Adjust this value based on your data
y_threshold = 0.01  # Adjust this value based on your data
z_threshold = 0.1  # Adjust this value based on your data


points = np.asarray(final_pcd.points)



# Create a mask for points that are at or below the height of the centroid
height_mask = points[:, 2] <= centroid[2] 

# Select points using the height mask
segment_points = points[height_mask]

# Create masks for points that are within the distance thresholds from the centroid
x_mask = np.abs(segment_points[:, 0] - centroid[0]) <= x_threshold
y_mask = np.abs(segment_points[:, 1] - centroid[1]) <= y_threshold

# Combine the masks for x and y
xy_mask = np.logical_and(x_mask, y_mask)

# Get the opposite of the xy_mask
inverse_xy_mask = ~xy_mask

# Select points using the inverse_xy_mask
final_segment_points = segment_points[inverse_xy_mask]

# Combine the final_segment_points with the points above the height of the centroid
final_points = np.vstack((points[~height_mask], final_segment_points))

# Convert the final points to a point cloud
final_pcd = o3d.geometry.PointCloud()
final_pcd.points = o3d.utility.Vector3dVector(final_points)

# Visualize
o3d.visualization.draw_geometries([final_pcd], window_name="Cleaned Point Cloud after Removing Far Points")


In [16]:
o3d.visualization.draw_geometries([outlier_cloud], window_name="Cleaned Point Cloud")